## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.30"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


In [ ]:
query = "SELECT DISTINCT(task) FROM dlc_table;"
df = pd.read_sql_query(query, conn)

print(df)

## Import plot_feature_barplot function 

In [ ]:
import Python_scripts.Data_analysis.plot_feature_barplot
import Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot

importlib.reload(Python_scripts.Data_analysis.plot_feature_barplot)
importlib.reload(Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot)

from Python_scripts.Data_analysis.plot_feature_barplot import plot_feature_barplot
from Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot import plot_feature_histogram_and_boxplot


## Extract IDs

In [ ]:
FoodOnly_saline_q = "SELECT id FROM dlc_table WHERE task='FoodOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(FoodOnly_saline_q, conn)
FoodOnly_saline_id = df['id'].to_list()

FoodOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='FoodOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(FoodOnly_ghrelin_q, conn)
FoodOnly_ghrelin_id = df['id'].to_list()


LightOnly_saline_q = "SELECT id FROM dlc_table WHERE task='LightOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(LightOnly_saline_q, conn)
LightOnly_saline_id = df['id'].to_list()

LightOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='LightOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(LightOnly_ghrelin_q, conn)
LightOnly_ghrelin_id = df['id'].to_list()


ToyOnly_saline_q = "SELECT id FROM dlc_table WHERE task='ToyOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(ToyOnly_saline_q, conn)
ToyOnly_saline_id = df['id'].to_list()

ToyOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(ToyOnly_ghrelin_q, conn)
ToyOnly_ghrelin_id = df['id'].to_list()


ToyLight_saline_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(ToyLight_saline_q, conn)
ToyLight_saline_id = df['id'].to_list()

ToyLight_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(ToyLight_ghrelin_q, conn)
ToyLight_ghrelin_id = df['id'].to_list()

## Feature barplot

In [ ]:
import matplotlib.pyplot as plt

ax = plot_feature_barplot(conn, FoodOnly_saline_id, FoodOnly_ghrelin_id, feature='mean_curv', 
                     group_labels=["Saline", "Ghrelin"])
ax.set_title('Food Only')
plt.show()


In [ ]:
ax = plot_feature_barplot(conn, LightOnly_saline_id, LightOnly_ghrelin_id, feature='mean_curv', 
                     group_labels=["Saline", "Ghrelin"])
ax.set_title('Light Only')
plt.show()


In [ ]:
ax = plot_feature_barplot(conn, ToyOnly_saline_id, ToyOnly_ghrelin_id, feature='mean_curv', 
                     group_labels=["Saline", "Ghrelin"])
ax.set_title('Toy Only')
plt.show()


In [ ]:
ax = plot_feature_barplot(conn, ToyLight_saline_id, ToyLight_ghrelin_id, feature='mean_curv', 
                     group_labels=["Saline", "Ghrelin"])
ax.set_title('Toy Light')
plt.show()


## Feature histogram and boxplot

In [ ]:
plot_feature_histogram_and_boxplot(conn, 'mean_curv')

## Feature distribution

In [ ]:
import Python_scripts.Data_analysis.plot_feature_distribution
importlib.reload(Python_scripts.Data_analysis.plot_feature_distribution)
from Python_scripts.Data_analysis.plot_feature_distribution import plot_feature_distribution


In [ ]:
print(type(FoodOnly_saline_id), type(FoodOnly_saline_id[0]))
print(type(FoodOnly_ghrelin_id), type(FoodOnly_ghrelin_id[0]))


In [ ]:
ax = plot_feature_distribution(conn, FoodOnly_saline_id, FoodOnly_ghrelin_id, feature='mean_curv', 
                              group_labels=['Saline', 'Ghrelin'], mode='pdf')

ax.set_title('Food Only')
plt.show()


## Summary statistics

In [ ]:
import importlib
import Python_scripts.Data_analysis.compute_array_feature_summaries
importlib.reload(Python_scripts.Data_analysis.compute_array_feature_summaries)
from Python_scripts.Data_analysis.compute_array_feature_summaries import compute_array_feature_summaries


In [ ]:
df_summary_sal = compute_array_feature_summaries(conn, LightOnly_saline_id, feature='distance')
df_sal = df_summary_sal.copy()
df_sal['group'] = 'Saline'

df_summary_ghr = compute_array_feature_summaries(conn, LightOnly_ghrelin_id, feature='distance')
df_ghr = df_summary_ghr.copy()
df_ghr['group'] = 'Ghrelin'

df_combined = pd.concat([df_sal, df_ghr], ignore_index=True)

df_combined

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
sns.boxplot(data=df_combined, x='group', y='curvature_entropy')
plt.title('Curvature Entropy')
plt.show()


In [ ]:
from scipy.stats import mannwhitneyu

for feature in ['curvature_mean', 'curvature_max', 'curvature_std', 'curvature_entropy', 
                'curvature_area', 'curvature_z_mean_abs']:
    u, p = mannwhitneyu(df_summary_sal[feature], df_summary_ghr[feature], alternative='two-sided')
    print(f"{feature}: p = {p:.4f}")


## Binned array stats

In [ ]:
import importlib
import Python_scripts.Data_analysis.compute_binned_curvature_stats
import Python_scripts.Data_analysis.plot_binned_curvature_comparison

importlib.reload(Python_scripts.Data_analysis.compute_binned_curvature_stats)
importlib.reload(Python_scripts.Data_analysis.plot_binned_curvature_comparison)

from Python_scripts.Data_analysis.compute_binned_curvature_stats import compute_binned_curvature_stats
from Python_scripts.Data_analysis.plot_binned_curvature_comparison import plot_binned_curvature_comparison


In [ ]:
df_sal_bins = compute_binned_curvature_stats(conn, ToyLight_saline_id, 'Saline')
df_ghr_bins = compute_binned_curvature_stats(conn, ToyLight_ghrelin_id, 'Ghrelin')

df_all_bins = pd.concat([df_sal_bins, df_ghr_bins], ignore_index=True)


In [ ]:
plot_binned_curvature_comparison(df_all_bins, feature='curvature_mean', title='ToyLight')
plot_binned_curvature_comparison(df_all_bins, feature='curvature_z_mean_abs', title='ToyLight')


In [ ]:
import importlib
import Python_scripts.Data_analysis.grid_search_curvature_hyperparams

importlib.reload(Python_scripts.Data_analysis.grid_search_curvature_hyperparams)

from Python_scripts.Data_analysis.grid_search_curvature_hyperparams import grid_search_curvature_hyperparams


In [ ]:
df_result = grid_search_curvature_hyperparams(
    conn,
    ToyLight_saline_id,
    ToyLight_ghrelin_id,
    window_values=[3, 5, 7, 9, 11, 13, 15, 17],
    speed_thresh_values=[1e-3, 5e-3, 1e-2, 2e-2, 5e-2]
)


In [ ]:
df_result

## Acceleration and jerk outliers

In [ ]:
query = "SELECT * FROM dlc_table LIMIT 1;"
df = pd.read_sql(query, conn)

print(df.columns)

In [ ]:
query = "SELECT DISTINCT(frame_rate) FROM dlc_table WHERE head_x IS NOT NULL;"
df = pd.read_sql(query, conn)

print(df['frame_rate'])